# Generate Full Disk images of GOES-R NetCDF files and upload them to GitLab database.

https://gitlab.com/adomakor412/goes-r_fd_image

In [1]:
import sys
from netCDF4 import Dataset
from datetime import datetime, timedelta
import numpy as np
import matplotlib.pyplot as plt
#Generates blank plots when saving unless called in same cell
#%matplotlib inline
from mpl_toolkits.basemap import Basemap
from pathlib import Path
from subprocess import Popen
import itertools

from pyproj import Proj
import pyproj
import xarray as xr
import matplotlib.pyplot as plt
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from pyresample import image, geometry
import metpy
import seaborn as sns
sns.set(style="darkgrid")
import pandas as pd
import statsmodels.api as sm
import os
import os.path as op
from glob import glob
from convertdate import gregorian, ordinal
from multiprocessing import Pool

import warnings
#warnings.filterwarnings('ignore')

import logging
logger = logging.getLogger()

/home/radomako/.conda/envs/adomako/lib/python3.9/site-packages/pyresample/bilinear/__init__.py:50: UserWarning: XArray and/or zarr not found, XArrayBilinearResampler won't be available.
  warnings.warn("XArray and/or zarr not found, XArrayBilinearResampler won't be available.")


## Paths

In [2]:
storage = '/scratch/radomako/goes-r_fd_image/'
fullDisk = '/arcdata/goes/grb/goes'

## Functions for unfiltered cloud mask

In [3]:
def Rad2BT(rad, planck_fk1, planck_fk2, planck_bc1, planck_bc2):
    """Radiances to Brightness Temprature (using black body equation)"""
    
    planck_fk1 = abs(planck_fk1)
    planck_fk2 = abs(planck_fk2)
    planck_bc1 = abs(planck_bc1)
    planck_bc2 = abs(planck_bc2)
    
    invRad = np.array(abs(rad))**(-1)
    arg = (invRad*planck_fk1) + 1.0
    T = (- planck_bc1+(planck_fk2 * (np.log(arg)**(-1))) )*(1/planck_bc2)
    return T

# def download(url,toPath, saveName):
#     cmd = [ 'wget ' + url +' -P ' + toPath +' -O '+ saveName]#if re.search('C07',url)
#     #print(cmd)
#     pid = Popen(cmd, shell=True)
#     pid.communicate()
#     return

## Load data

In [4]:
Sat = [16,17]
#band = range(7,17)
year = [2021]
month = list(range(1,2))#list(range(1,3))
day = list(range(1,2))#list(range(1,32))
hour = list(range(24))

In [5]:
search = list(itertools.product(Sat,\
        #band,\
        year,\
        month,\
        day,\
        hour))

## Execution

In [14]:
def execute(searchBandTuple):
    chunk, band = searchBandTuple
    filelog = open('log_FD_image.txt','w')
    #for SS, bb, yyyy, mm, dd, hr in chunk:
    #SS, bb, yyyy, mm, dd, hr = chunk
    SS, yyyy, mm, dd, hr = chunk
    bb = band
    SS, bb, yyyy, mm, dd, hr = \
        str(SS).zfill(2),\
        str(bb).zfill(2),\
        str(yyyy).zfill(4),\
        str(mm).zfill(2),\
        str(dd).zfill(2),\
        str(hr).zfill(2)

    calDate = gregorian.date(int(yyyy), int(mm), int(dd))
    DDD = ordinal.from_gregorian(calDate.year, calDate.month, calDate.day)[1]
    DDD = str(DDD).zfill(3)

    '''Create Directories'''
    cond2 = os.path.exists(f'{storage}/{yyyy}')
    cond3 = os.path.exists(f'{storage}/{yyyy}/{DDD}')
    #cond4 = os.path.exists(f'{storage}/{yyyy}/{DDD}/{bb}')

    if not cond2:
        #KEEP CONDITION AS A DYNAMIC F-STRING OTHERWISE VARIABLE IS STATIC, utilize loop
        #make di
            #Create Directoriesrectory per year
        cmd = [f'mkdir {storage}/{yyyy}']
        pid = Popen(cmd, shell=True) 
        pid.communicate()

    if not cond3:
        #KEEP CONDITION AS A DYNAMIC F-STRING OTHERWISE VARIABLE IS STATIC, utilize loop
        #make directory per year
        cmd = [f'mkdir {storage}/{yyyy}/{DDD}']
        pid = Popen(cmd, shell=True) 
        pid.communicate()
        
#     if not cond4:
#         #KEEP CONDITION AS A DYNAMIC F-STRING OTHERWISE VARIABLE IS STATIC, utilize loop
#         #make directory per year
#         cmd = [f'mkdir {storage}/{yyyy}/{DDD}/{bb}']
#         pid = Popen(cmd, shell=True) 
#         pid.communicate()

    try:
        #Not a cd command, use absolute path
        #netcdf = glob.glob(f'~/arcdata/goes/grb/goes{SS}/{yyyy}/{yyyy}_{mm}_{dd}_{DDD}/abi/L1b/RadF/')
        netcdf = glob(f'{fullDisk}'+f'{SS}/{yyyy}/{yyyy}_{mm}_{dd}_{DDD}/abi/L1b/RadF/*.nc')
            
        print(netcdf, file=filelog)

        for file in netcdf:
#             if len(netcdf)==0:
#                 print('\n NO SOURCE DATA \n', file=filelog)
#                 continue
            #print(file)
            i = file.find('_s')#+2
            HH = file[i+9: i+11]
            MM = file[i+11: i+13]
            ss = file[i+13: i+15]
            
            my_dpi = 192#96#
            #resolution = 5424
            downsize_x = 224
            downsize_y = 224
            
            savePath = op.join(storage)
            
#             naming = f'{yyyy}/{DDD}/{bb}/FD_goes{SS}_B{bb}_{yyyy}-{mm}-{dd}_{DDD}_{HH}{MM}{ss}'\
#                 + 'UTC_5424x5424_plain.png'
            
            naming = f'{yyyy}/{DDD}/FD_goes{SS}_B{bb}_{yyyy}-{mm}-{dd}_{DDD}_{HH}{MM}{ss}'\
                + f'UTC_{downsize_x}x{downsize_y}_BW.jpg'
    
            #cond5 = os.path.exists(f'{storage}/{yyyy}/{DDD}/{bb}/{naming}')
            cond5 = os.path.exists(f'{storage}/{yyyy}/{DDD}/{naming}')
            
            if cond5:
                continue
            '''
            https://www.infobyip.com/detectmonitordpi.php
            stackoverflow.com/questions/13714454/specifying-and-saving-a-figure-with-exact-size-in-pixels
            '''
            
            
            GOES_R = xr.open_dataset(file)
            GOES_image = GOES_R['Rad']

            planck_fk1 = float(GOES_R['planck_fk1'].data)
            planck_fk2 = float(GOES_R['planck_fk2'].data) 
            planck_bc1 = float(GOES_R['planck_bc1'].data)                       
            planck_bc2 = float(GOES_R['planck_bc2'].data)
            '''
            Need not convert projection: using full image, no pyproj interpolation of region of interest
            '''
            
            Kelvin_GOES = Rad2BT(GOES_image, \
                                 planck_fk1, \
                                 planck_fk2, \
                                 planck_bc1, \
                                 planck_bc2)
            
            '''Call pixelage before rendering'''
            plt.figure(figsize=(downsize_x/my_dpi, downsize_y/my_dpi), dpi=my_dpi)
            
            plt.grid(None)
            plt.axis('off')
#             title = f'Brightness Temperature in Kelvin \n Full Disk GOES-{SS} Image Band{bb}'
#             ttl = plt.title(f"{title}\n Date {mm}-{dd}, Year {yyyy}, \n Day {DDD}, Time {HH}:{MM}:{ss}",\
#                        fontsize="x-large", fontweight='bold', pad = 6.0)
#             ttl.set_position([.5, 1.05])
            fig1=plt.imshow(Kelvin_GOES, cmap='Greys',vmin=180,vmax=300)#light to dark
            '''cmap = 'gray' reverses black and white
                https://matplotlib.org/stable/tutorials/colors/colormaps.html
            '''
            arr = fig1.get_array()
#             cbar = plt.colorbar()
#             cbar.ax.set_ylabel('Kelvin')
            saveName = op.join(savePath,naming)
            if not os.path.isfile(saveName):
                #plt.imsave( op.join(storage,naming), arr = arr)
                fig1.figure.savefig(saveName)
                cmd = [ 'convert '+ saveName +' -colorspace Gray '+ saveName]#if re.search('C07',url)
                pid = Popen(cmd, shell=True)
                pid.communicate()
            
            naming = f'{yyyy}/{DDD}/' + \
                f'FD_goes{SS}_B{bb}_{yyyy}-{mm}-{dd}_{DDD}_{HH}{MM}{ss}UTC_{downsize_x}x{downsize_y}.jpg'  
#             cbar.remove()    
#             title = f'Brightness Temperature in Kelvin \n Full Disk GOES-{SS} Image Band{bb}'
#             ttl = plt.title(f"{title}\n Date {mm}-{dd}, Year {yyyy}, \n Day {DDD}, Time {HH}:{MM}:{ss}",\
#                        fontsize="x-large", fontweight='bold', pad = 6.0)
#             ttl.set_position([.5, 1.05])
            fig1 = plt.imshow(Kelvin_GOES, vmin=180, vmax=300, cmap='plasma')
            arr = fig1.get_array()
#             cbar = plt.colorbar()
#             cbar.ax.set_ylabel('Kelvin')
            plt.grid(None) 
            plt.axis('off')

#             naming = f'{yyyy}/{DDD}/{bb}/' + \
#                 f'FD_goes{SS}_B{bb}_{yyyy}-{mm}-{dd}_{DDD}_{HH}{MM}{ss}UTC_5424x5424.png'
                
            if not os.path.isfile(op.join(storage,naming)):
                #plt.imsave(op.join(storage,naming), arr = arr)
                fig1.figure.savefig(op.join(savePath,naming))
            plt.close('all')
            GOES_R.close()

    except ValueError as e:
        print('THERE IS AN ERROR')
        print('\n', file = filelog)
        print(file, file = filelog)
        print(e, file = filelog)
        print('\n', file = filelog)

    filelog.close()
    return

#### This multiprocessing may generate warning of too many plots open before closing them all

In [ ]:
agents = 1
chunksize = 3

# use number of bands as agents
with Pool(processes = agents) as pool:
    #pool.map(execute, search, chunksize=None)
    args = [(hour, band) for hour in search for band in range(8,17)]
    pool.map(execute, args, chunksize = chunksize)